In [1]:
import pandas as pd
import numpy as np
import datetime
import sys, os
import glob
import time
import fnmatch
import os
import datetime
import xlwings as xw
from datetime import date
start_time = time.time()

a=fnmatch.filter(os.listdir('.'), 'ZCM UPLOADS*')
a=a[0]

a

'ZCM UPLOADS 6.13 v2.xlsx'

# OH TX2 BEFORE JULY

In [2]:
zcmuploads = pd.read_excel(a,dtype={'*Zebra PN': str, '*Zebra Site': str,'*Supplier ID': str}, sheet_name='WEEKLY- Net Reqs_tx2')

zcmuploads['*Date'] = pd.to_datetime(zcmuploads['*Date']).dt.date
zcmuploads.head()
print(len(zcmuploads),'NUMBER OF ROWS')
#get date of june 30 2022
july1 = datetime.date(2022, 7, 3)

#identify infor before july
mask1 = (zcmuploads['*Date'] <= july1)
zcmuploads['month'] = np.where(mask1, 'beforejuly','afterjuly')
zcmuploads.head()

#zcmuploads = zcmuploads[(zcmuploads['*Transaction Type'] == 'ONHANDWK') | (zcmuploads['*Transaction Type'] == 'Net Requirements') | 
                       #(zcmuploads['*Transaction Type'] == 'GROSSFRCTWK') | 
                       #(zcmuploads['*Transaction Type'] == 'ONHANDDLY')] #delete extra rows

zcmuploads.drop(labels=['Flex String PIT 1','Flex String PIT 2','Flex String PIT 3','Supplier PN'], axis=1, inplace = True)
zcmuploads.rename(columns={'*Quantity': 'Quantity'},inplace=True)
zcm_onhand_dly= zcmuploads[(zcmuploads['*Transaction Type']=='GROSSFRCTWK')]
zcm_onhand_dly

#zcm_onhand_dly.loc[zcm_onhand_dly['*Zebra PN'] == "8617"] #no forecast THRMP22CC370187 #missing ok, why doble qty?
#for ohtx2 we only need forecast for TX2
zcm_onhand_dlytx2=zcm_onhand_dly.copy()

#zcmuploads.head()
#sum of forecast JUST FOR TX2
zcmuploadspiv=zcm_onhand_dlytx2.pivot_table(index=('*Zebra PN','*Supplier ID'), columns='month', aggfunc='sum').fillna(0)
print(len(zcmuploadspiv),'measures of forecast')
#zcmuploadspiv['*Transaction Type'] ='ONHANDWK'
zcmuploadspiv.head()

#NUMBER OF ROWS FOR TX2
zcm_onhand_now= zcmuploads[(zcmuploads['*Transaction Type']=='ONHANDWK')] #edit
#zcm_onhand_now
#sum of oh after/before july
zcm_oh_now_piv=zcm_onhand_now.pivot_table(index=('*Zebra PN','*Supplier ID'), columns='month', aggfunc='sum').fillna(0)
print(len(zcm_oh_now_piv))
zcm_oh_now_piv.head()

#as long as PN is in TX2 OH input it should be in OH ouput as we'll wip information
oh_to_upload_tx2= zcmuploads[(zcmuploads['*Transaction Type']=='ONHANDWK') & (zcmuploads['*Zebra Site']=='TX2')]
oh_to_upload_tx2=oh_to_upload_tx2.drop_duplicates(subset=('*Zebra PN','*Supplier ID'),keep='first')
oh_to_upload_tx2= oh_to_upload_tx2.drop(['#*Zebra','*Zebra Site','Quantity','*Date','*Transaction Type','month'], axis = 1)

oh_to_upload_tx2.head()

#assign corresponding OH(forecast piv) TO rows that will be uploaded
oh_to_upload_tx2 = pd.merge(oh_to_upload_tx2,zcmuploadspiv, on =['*Zebra PN','*Supplier ID'],how='left')
oh_to_upload_tx2[('Quantity','afterjuly')]=oh_to_upload_tx2[('Quantity','afterjuly')].fillna(0)
oh_to_upload_tx2[('Quantity','beforejuly')]=oh_to_upload_tx2[('Quantity','beforejuly')].fillna(0)
oh_to_upload_tx2.head()

oh_to_upload_tx2.rename(columns={('Quantity','afterjuly'):'Fcst_after_july'},inplace=True)
oh_to_upload_tx2.rename(columns={('Quantity','beforejuly'):'Fcst_before_july'},inplace=True)
#net_req_tx4.rename(columns={'Quantity': 'Forecast'},inplace=True)
oh_to_upload_tx2['*Date']=date.today()
oh_to_upload_tx2['*Zebra Site'] = 'TX2'

oh_to_upload_tx2= pd.merge(oh_to_upload_tx2,zcm_oh_now_piv,on =['*Zebra PN','*Supplier ID'],how='left')
oh_to_upload_tx2.head()

oh_to_upload_tx2[('Quantity', 'beforejuly')] = oh_to_upload_tx2[('Quantity', 'beforejuly')].fillna(0)
oh_to_upload_tx2.rename(columns={('Quantity','beforejuly'):'OH_beforeuly'},inplace=True)
oh_to_upload_tx2.head()

#zcm_onhand_dly131.loc[zcm_onhand_dly13['*Zebra PN'] == "P1031365-045"]
sells_orders= zcmuploads[(zcmuploads['*Transaction Type']=='BOOKEDSODLY') & (zcmuploads['month']=='afterjuly')]
sells_orders

#sum of oh after/before july
sells_orders_piv=sells_orders.pivot_table(index=('*Zebra PN','*Supplier ID'), aggfunc='sum').fillna(0)
print(len(sells_orders_piv),'NBO number of rows')
sells_orders_piv.head()

so= sells_orders_piv.rename(columns={'Quantity':'SO'})
so

irs= zcmuploads[(zcmuploads['*Transaction Type']=='INTERNALREQDLY') & (zcmuploads['month']=='afterjuly')]
irs

#sum of oh after/before july
irs_piv=irs.pivot_table(index=('*Zebra PN','*Supplier ID'), aggfunc='sum').fillna(0)
print(len(irs),'NBO number of rows')
irs.head()
irs_1= irs_piv.rename(columns={'Quantity':'IRS'})
irs_1

oh_to_upload_tx2_so = pd.merge(oh_to_upload_tx2,so,on =['*Zebra PN','*Supplier ID'],how='left')
oh_to_upload_tx2_so['SO']=oh_to_upload_tx2_so['SO'].fillna(0)
oh_to_upload_tx2_so

oh_to_upload_tx2_so_irs = pd.merge(oh_to_upload_tx2_so,irs_1,on =['*Zebra PN','*Supplier ID'],how='left')
oh_to_upload_tx2_so_irs['IRS']=oh_to_upload_tx2_so_irs['IRS'].fillna(0)
oh_to_upload_tx2_so_irs

oh_to_upload_tx2_so_irs['Demand'] = oh_to_upload_tx2_so_irs['Fcst_before_july'] + oh_to_upload_tx2_so_irs['SO'] + oh_to_upload_tx2_so_irs['IRS']
oh_to_upload_tx2_so_irs

#if OH less than total forecast before july, then put all oh in TX2
oh_to_upload_tx2_so_irs['Quantity']=  np.where(oh_to_upload_tx2_so_irs['Demand'] < oh_to_upload_tx2_so_irs['OH_beforeuly'], 
                                         oh_to_upload_tx2_so_irs['Demand'],oh_to_upload_tx2_so_irs['OH_beforeuly'])
oh_to_upload_tx2_so_irs.head()

#SET OH AS 0 for PN with no forecast beforejuly
oh_to_upload_tx2_so_irs['Quantity']=  np.where(oh_to_upload_tx2_so_irs['Demand'] == 0,0,oh_to_upload_tx2_so_irs['Quantity'])

oh_to_upload_tx2_so_irs['Quantity']=oh_to_upload_tx2_so_irs['Quantity'].fillna(0)

oh_to_upload_tx2_so_irs

37825 NUMBER OF ROWS
4767 measures of forecast
6313
2329 NBO number of rows
11 NBO number of rows


C:\Users\MJ7255\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,*Supplier ID,*Zebra PN,Fcst_after_july,Fcst_before_july,*Date,*Zebra Site,OH_beforeuly,SO,IRS,Demand,Quantity
0,25738,01970-058-3,0.0,0.0,2022-06-13,TX2,2031.0,0.0,0.0,0.0,0.0
1,25738,P1037750-041,0.0,0.0,2022-06-13,TX2,14.0,0.0,0.0,0.0,0.0
2,24075,TC77HL-7ME24BG-NA24,0.0,0.0,2022-06-13,TX2,10.0,0.0,0.0,0.0,0.0
3,25738,P1112750-074,0.0,0.0,2022-06-13,TX2,1.0,0.0,0.0,0.0,0.0
4,36849,RFD9031-G60G700-US,0.0,140.0,2022-06-13,TX2,1.0,0.0,0.0,140.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
6308,25738,ZE50043-R010000Z,0.0,139.0,2022-06-13,TX2,1.0,13.0,0.0,152.0,1.0
6309,36849,ZLED-XS40CW-0000,0.0,10.0,2022-06-13,TX2,21.0,0.0,0.0,10.0,10.0
6310,36208,ZP450-0501-0000A,0.0,305.0,2022-06-13,TX2,1015.0,70.0,0.0,375.0,375.0
6311,25738,105934-130,0.0,46.0,2022-06-13,TX2,22.0,0.0,0.0,46.0,22.0


In [3]:
strings = time.strftime("%m.%d.%Y")
stringstx2= 'OHTX2_WEEKLY ' + strings 
b=("%s.xlsx" % stringstx2)

with pd.ExcelWriter(b) as writer:  
       oh_to_upload_tx2_so_irs.to_excel(writer, index= False)

# OH TX3 BEFORE JULY

* TX2, TX3 THERE WILL BE NO UPLOADS FOR NET REQ

# OH  TX4 (AFTER JULY)

In [40]:
zcm_onhand_now= zcmuploads[(zcmuploads['*Transaction Type']=='ONHANDWK')] #edit
zcm_onhand_now
#sum of oh after july
zcm_oh_now_piv=zcm_onhand_now.pivot_table(index=('*Zebra PN','*Supplier ID'), columns='month', aggfunc='sum').fillna(0)
print(len(zcm_oh_now_piv))
zcm_oh_now_piv.head()

#one part number per quarter, delete column transaction type so if it coccurs on either oh prev july or frcst the row will be consider
zcm_net_req = zcm_onhand_now.drop(['*Transaction Type'], axis=1)
zcm_net_req =zcm_net_req.drop_duplicates(subset=('*Zebra PN','month','*Supplier ID'),keep='first')
zcm_net_req.drop(labels=['Quantity','month','*Date','*Zebra Site'], axis=1, inplace = True)
print(zcm_net_req.shape,'total rows for ohtx4')
zcm_net_req.head()

#FIND ROW OF PARTICULAR PART NUMBER PN
#zcm_net_req.loc[zcm_net_req['*Zebra PN'] == "P1076537"]

#get the total sum of current OH before and after july as well as newOHuploaded in TX2 per row
zcm_net_req0=pd.merge(zcm_net_req,oh_to_upload_tx2_so_irs, on =['*Zebra PN','*Supplier ID'],how='left')
zcm_net_req0.head()

#zcm_net_req0[('Quantity', 'afterjuly')] = zcm_net_req0[('Quantity', 'afterjuly')].fillna(0)
zcm_net_req0['Quantity'] = zcm_net_req0['Quantity'].fillna(0)
print(len(zcm_net_req0))
zcm_net_req0

zcm_net_req0['OHTX4'] = zcm_net_req0['OH_beforeuly'] - zcm_net_req0['Quantity']
zcm_net_req0

6313
(6313, 3) total rows for ohtx4
6313


,#*Zebra,*Supplier ID,*Zebra PN,Fcst_after_july,Fcst_before_july,*Date,*Zebra Site,OH_beforeuly,SO,IRS,Demand,Quantity,OHTX4
0,Zebra,25738,01970-058-3,0.0,0.0,2022-06-12,TX2,2031.0,0.0,0.0,0.0,0.0,2031.0
1,Zebra,25738,P1037750-041,0.0,0.0,2022-06-12,TX2,14.0,0.0,0.0,0.0,0.0,14.0
2,Zebra,24075,TC77HL-7ME24BG-NA24,0.0,0.0,2022-06-12,TX2,10.0,0.0,0.0,0.0,0.0,10.0
3,Zebra,25738,P1112750-074,0.0,0.0,2022-06-12,TX2,1.0,0.0,0.0,0.0,0.0,1.0
4,Zebra,36849,RFD9031-G60G700-US,0.0,212.0,2022-06-12,TX2,1.0,0.0,0.0,212.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6308,Zebra,25738,ZE50043-R010000Z,13.0,278.0,2022-06-12,TX2,1.0,13.0,0.0,291.0,1.0,0.0
6309,Zebra,36849,ZLED-XS40CW-0000,0.0,0.0,2022-06-12,TX2,21.0,0.0,0.0,0.0,0.0,21.0
6310,Zebra,36208,ZP450-0501-0000A,0.0,0.0,2022-06-12,TX2,1015.0,70.0,0.0,70.0,70.0,945.0
6311,Zebra,25738,105934-130,0.0,24.0,2022-06-12,TX2,22.0,0.0,0.0,24.0,22.0,0.0


In [48]:
new_ohtx4= zcm_net_req0.copy()
new_ohtx4.drop(labels=['Fcst_before_july','OH_beforeuly','SO','IRS','Demand','Quantity','#*Zebra'], axis=1, inplace = True)
new_ohtx4.rename(columns={'OHTX4':'Quantity'}, inplace= True)
new_ohtx4['*Transaction Type']='OHWEEKLY'
new_ohtx4['*Zebra Site']='TX4'
new_ohtx4.head()

,*Supplier ID,*Zebra PN,Fcst_after_july,*Date,*Zebra Site,Quantity,*Transaction Type
0,25738,01970-058-3,0.0,2022-06-12,TX4,2031.0,OHWEEKLY
1,25738,P1037750-041,0.0,2022-06-12,TX4,14.0,OHWEEKLY
2,24075,TC77HL-7ME24BG-NA24,0.0,2022-06-12,TX4,10.0,OHWEEKLY
3,25738,P1112750-074,0.0,2022-06-12,TX4,1.0,OHWEEKLY
4,36849,RFD9031-G60G700-US,0.0,2022-06-12,TX4,0.0,OHWEEKLY


In [47]:
#Add info already in OHTX4 IR, OH in TX4
ohtx4_prev_week = pd.read_excel(a,dtype={'#ITEM': str, '#SITE_NAME': str,'#SUPPLIER_NUMBER': str}, sheet_name='OHTX4')
ohtx4_prev_week['*Date'] = pd.to_datetime(ohtx4_prev_week['#SCHEDULE_DOCK_DATE']).dt.date
ohtx4_prev_week.drop(labels=['#CUSTOMER','#LOCATION_CODE','#ATTRIBUTE1','#ATTRIBUTE2','#ATTRIBUTE3','#CUSTOMER','#SCHEDULE_DOCK_DATE'], axis=1, inplace = True)
ohtx4_prev_week.rename(columns={'#ITEM':'*Zebra PN','#SUPPLIER_NUMBER':'*Supplier ID','Quantity':'OLD_OH_TX4'},inplace=True)

ohtx4_prev_week.rename(columns={'#TRANSACTION_TYPE':'*Transaction Type',
                                '#QUANTITY':'Quantity','#SITE_NAME':'*Zebra Site'},inplace=True)
ohtx4_prev_week

##IRtx4_prev_week = ohtx4_prev_week[ohtx4_prev_week['*Transaction Type']!= 'ONHANDWK']
#IRtx4_prev_week
#new_ohtx41 =new_ohtx4.append(IRtx4_prev_week,ignore_index =  True)
#new_ohtx41
#to sum oh already in TX4

,*Supplier ID,*Zebra PN,*Zebra Site,Quantity,*Transaction Type,*Date
0,18048,CHG-MPP-BEPSAC2-01,TX4,1589,ONHANDWK,2022-06-13
1,23302,MC930P-GSCFG4NA-03,TX4,1140,ONHANDWK,2022-06-13
2,25738,P1031365-042,TX4,1289,ONHANDWK,2022-06-13
3,13451,CRD-MPM-5SCHGUS1-01,TX4,1624,ONHANDWK,2022-06-13
4,15012,P1052506,TX4,50,ONHANDWK,2022-06-13
5,36208,AS-000255-08P,TX4,15854,ONHANDWK,2022-06-13
6,15012,P1031096,TX4,1630,ONHANDWK,2022-06-13
7,13451,SAC-MPP-1BCHGUS1-01,TX4,18,ONHANDWK,2022-06-13
8,12666,AT16305-8,TX4,1440,ONHANDWK,2022-06-13
9,24260,MP7011-LNSLM00NN,TX4,98,ONHANDWK,2022-06-13


In [45]:
strings = time.strftime("%m.%d.%Y")
stringstx4= 'OHTX4_WEEKLY ' + strings 
b=("%s.xlsx" % stringstx4)

with pd.ExcelWriter(b) as writer:  
       new_ohtx41.to_excel(writer, index= False)

# NET REQ TX4

In [2]:
zcmuploads_nr = pd.read_excel(a,dtype={'*Zebra PN': str, '*Zebra Site': str,'*Supplier ID': str}, sheet_name='NR_TX4')
zcmuploads_nr['*Date'] = pd.to_datetime(zcmuploads_nr['*Date']).dt.date
zcmuploads_nr.head()

#identify infor before july
#mask1 = (zcmuploads_nr['*Date'] <= july1)
#zcmuploads_nr['month'] = np.where(mask1, 'beforejuly','afterjuly')
#zcmuploads_nr.head()

zcmuploads_nr.drop(labels=['Flex String PIT 1','Flex String PIT 2','Flex String PIT 3','Supplier PN','#*Zebra'], axis=1, inplace = True)
zcmuploads_nr.rename(columns={'*Quantity': 'Quantity'},inplace=True)
#zcmuploads_nr= zcmuploads_nr[(zcmuploads_nr['month']=='beforejuly')]
print(len(zcmuploads_nr))
zcmuploads_nr.head()

216307


,*Supplier ID,*Zebra PN,*Zebra Site,*Transaction Type,Quantity,*Date
0,12257,P1047939,TX4,GROSSFRCTWK,2,2022-11-04
1,12257,P1047939,TX4,GROSSFRCTWK,1,2022-09-30
2,12257,P1047939,TX4,GROSSFRCTWK,2,2023-06-30
3,12257,P1036802,TX4,GROSSFRCTWK,2,2022-09-09
4,12257,P1062978,TX4,GROSSFRCTWK,3,2023-04-21


In [6]:
zcmuploads_so = pd.read_excel(a,dtype={'*Zebra PN': str, '*Zebra Site': str,'*Supplier ID': str}, sheet_name='SO_TX4')

zcmuploads_so['*Date'] = pd.to_datetime(zcmuploads_so['#SCHEDULE_DOCK_DATE']).dt.date
zcmuploads_so.head()

zcmuploads_so.drop(labels=['#ATTRIBUTE1','#ATTRIBUTE2','#ATTRIBUTE3','#CUSTOMER',
                          '#LOCATION_CODE','#SCHEDULE_DOCK_DATE'], axis=1, inplace = True)
zcmuploads_so.rename(columns={'#QUANTITY': 'Quantity','#SITE_NAME':'*Zebra Site','#TRANSACTION_TYPE':'*Transaction Type'},inplace=True)

print(len(zcmuploads_so))
zcmuploads_so.head()

258


,*Supplier ID,*Zebra PN,*Zebra Site,Quantity,*Transaction Type,*Date
0,36208,ZT61043-T010100Z,TX4,24,BOOKEDSODLY,2023-01-04
1,36849,MS4717-LU000R,TX4,400,BOOKEDSODLY,2022-11-10
2,24066,STND-WS0060C-04,TX4,2000,BOOKEDSODLY,2022-11-24
3,36208,ZT51042-T210000Z,TX4,96,BOOKEDSODLY,2023-02-03
4,36208,ZD22042-T0AG00EZ,TX4,600,BOOKEDSODLY,2022-12-28


In [7]:
nr_so= zcmuploads_nr.append(zcmuploads_so,ignore_index=True)
nr_so

,*Supplier ID,*Zebra PN,*Zebra Site,*Transaction Type,Quantity,*Date
0,12257,P1047939,TX4,GROSSFRCTWK,2,2022-11-04
1,12257,P1047939,TX4,GROSSFRCTWK,1,2022-09-30
2,12257,P1047939,TX4,GROSSFRCTWK,2,2023-06-30
3,12257,P1036802,TX4,GROSSFRCTWK,2,2022-09-09
4,12257,P1062978,TX4,GROSSFRCTWK,3,2023-04-21
...,...,...,...,...,...,...
216560,36208,ZT41142-T310000Z,TX4,BOOKEDSODLY,50,2022-12-06
216561,36208,ZQ62-HUWA000-00,TX4,BOOKEDSODLY,25,2022-10-28
216562,36208,ZT51042-T010000Z,TX4,BOOKEDSODLY,72,2022-12-02
216563,36208,ZT61043-T010100Z,TX4,BOOKEDSODLY,48,2022-11-08


In [41]:
nr_so_oh= nr_so.append(new_ohtx41,ignore_index=True)
nr_so_oh.drop(labels=['*Zebra Site'], axis=1, inplace = True)
nr_so_oh

,*Supplier ID,*Zebra PN,*Transaction Type,Quantity,*Date
0,12257,P1047939,GROSSFRCTWK,2.0,2022-11-04
1,12257,P1047939,GROSSFRCTWK,1.0,2022-09-30
2,12257,P1047939,GROSSFRCTWK,2.0,2023-06-30
3,12257,P1036802,GROSSFRCTWK,2.0,2022-09-09
4,12257,P1062978,GROSSFRCTWK,3.0,2023-04-21
...,...,...,...,...,...
222881,24260,SP5504-SR01000SSEU,INTERNALREQDLY,7.0,2022-06-05
222882,23302,MC930P-GSCFG4NA-03,INTERNALREQDLY,1450.0,2022-08-08
222883,15012,P1052506,INTERNALREQDLY,50.0,2022-08-01
222884,25738,P1031365-042,INTERNALREQDLY,1289.0,2022-08-01


In [42]:
nr_so_oh['*Transaction Type'].unique()

array(['GROSSFRCTWK', 'BOOKEDSODLY', 'OHWEEKLY', 'INTERNALREQDLY'],
      dtype=object)

In [59]:
main_tab_tx4=nr_so_oh.pivot_table(index=('*Zebra PN','*Supplier ID','*Date'), columns='*Transaction Type', aggfunc='sum',values ='Quantity').fillna(0)
print(len(main_tab_tx4),'main_df')
main_tab_tx4.head()
#decide wether fcst or booked so we'll consider ra
main_tab_tx4['SO_or_FCST']=np.where(main_tab_tx4['BOOKEDSODLY']>main_tab_tx4['GROSSFRCTWK'],main_tab_tx4['BOOKEDSODLY'],
                                    main_tab_tx4['GROSSFRCTWK'])

main_tab_tx4=main_tab_tx4.reset_index()

In [ ]:
main_tab_tx4['OH_covering_Demand']=np.where(main_tab_tx4['OHWEEKLY']>main_tab_tx4['SO_or_FCST'],main_tab_tx4['BOOKEDSODLY'],
                                    main_tab_tx4['GROSSFRCTWK'])

In [60]:
main_tab_tx4

*Transaction Type,*Zebra PN,*Supplier ID,*Date,BOOKEDSODLY,GROSSFRCTWK,INTERNALREQDLY,OHWEEKLY,SO_or_FCST
0,00375-000,25779,2022-06-07,0.0,0.0,0.0,100.0,0.0
1,01473-000,14160,2022-06-07,0.0,0.0,0.0,539.0,0.0
2,01807-000,25738,2022-06-07,0.0,0.0,0.0,791.0,0.0
3,01843R,13094,2022-06-07,0.0,0.0,0.0,61.0,0.0
4,01890-300,25738,2022-06-07,0.0,0.0,0.0,2288.0,0.0
...,...,...,...,...,...,...,...,...
222854,ZT62063-T210200Z,25738,2023-10-20,0.0,4.0,0.0,0.0,4.0
222855,ZT62063-T210200Z,25738,2023-10-27,0.0,3.0,0.0,0.0,3.0
222856,ZT62063-T210200Z,25738,2023-11-03,0.0,6.0,0.0,0.0,6.0
222857,ZT62063-T210200Z,25738,2023-11-10,0.0,5.0,0.0,0.0,5.0


In [163]:
zcmuploads_ss = pd.read_csv('Global-Safety Stock Items with Current Date (OBI) (1).csv',dtype={'Item': str, 'Inventory Org Name':str})

zcmuploads_ss.head()

zcmuploads_ss.drop(labels=['Product Class Description','Product Line Description','Product Family Description',
                          'Item Description'], axis=1, inplace = True)
zcmuploads_ss.rename(columns={'Item': '*Zebra PN'},inplace=True)

zcmuploads_ss.rename(columns={'Inventory Org Name': '*Zebra Site'},inplace=True)

zcmuploads_ss= zcmuploads_ss[(zcmuploads_ss['*Zebra Site']=='TX4-Kenosha Distribution Center (Hardware)')]
zcmuploads_ss['Plant Safety Stock'].fillna(0, inplace=True)

zcmuploads_ss.drop(labels=['*Zebra Site'], axis=1, inplace = True)
zcmuploads_ss.head()

,*Zebra PN,Plant Safety Stock
391013,15-08585-01,0.0
391014,15-81327-01,0.0
391015,15-81368-01,0.0
391016,15-81378-01,0.0
391017,15-81378-02,0.0


In [164]:
net_req_tx4_1= pd.merge(net_req_tx4,zcmuploads_ss, on =['*Zebra PN'],how='left')
net_req_tx4_1['Plant Safety Stock'].fillna(0, inplace=True)
net_req_tx4_1['Demand']=net_req_tx4_1['FCST or SO'] +net_req_tx4_1['Plant Safety Stock']
net_req_tx4_1.head()

,#*Zebra,*Supplier ID,*Zebra PN,*Zebra Site,*Transaction Type,Quantity,*Date,month,SO,FCST or SO,Plant Safety Stock,Demand
0,Zebra,12272,CBA-UF6-C12ZAR,TX4,GROSSFRCTWK,473,2022-06-13,beforejuly,0.0,473.0,0.0,473.0
1,Zebra,12272,CBA-UF6-C12ZAR,TX4,GROSSFRCTWK,473,2022-06-05,beforejuly,0.0,473.0,0.0,473.0
2,Zebra,12272,CBL-TC7X-USB1-01,TX4,GROSSFRCTWK,20,2022-06-13,beforejuly,0.0,20.0,0.0,20.0
3,Zebra,12272,SAC-ET5X-4PPK1-01,TX4,GROSSFRCTWK,26,2022-06-05,beforejuly,0.0,26.0,0.0,26.0
4,Zebra,12272,CBA-UF6-C12ZAR,TX4,GROSSFRCTWK,473,2022-06-06,beforejuly,0.0,473.0,0.0,473.0


In [153]:
#get oh uploaded for tx4
net_req_tx4_2= pd.merge(net_req_tx4_1,zcm_net_req0, on =['*Zebra PN','*Supplier ID'],how='left')
net_req_tx4_2['OHTX4'].fillna(0, inplace=True)
net_req_tx4_2.head()

,#*Zebra_x,*Supplier ID,*Zebra PN,*Zebra Site_x,*Transaction Type,Quantity_x,*Date_x,month,SO_x,FCST or SO,...,#*Zebra_y,Fcst_before_july,*Date_y,*Zebra Site_y,OH_beforeuly,SO_y,IRS,Demand_y,Quantity_y,OHTX4
0,Zebra,12272,CBA-UF6-C12ZAR,TX4,GROSSFRCTWK,3,2022-06-13,beforejuly,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,Zebra,12272,CBA-UF6-C12ZAR,TX4,GROSSFRCTWK,3,2022-06-05,beforejuly,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,Zebra,12272,CBL-TC7X-USB1-01,TX4,GROSSFRCTWK,3,2022-06-13,beforejuly,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,Zebra,12272,SAC-ET5X-4PPK1-01,TX4,GROSSFRCTWK,3,2022-06-05,beforejuly,0.0,3.0,...,Zebra,0.0,2022-06-05,TX2,9.0,0.0,0.0,0.0,0.0,9.0
4,Zebra,12272,CBA-UF6-C12ZAR,TX4,GROSSFRCTWK,3,2022-06-06,beforejuly,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [154]:
net_req_tx4_2['Net req'] = net_req_tx4_2['Demand_x'] - net_req_tx4_2['OHTX4']
#IF negative net req means oh bigger than forecast, so no more forecast=0, otherwise upload forecast needed
net_req_tx4_2['Net req'] = np.where(net_req_tx4_2['Net req'] < 0, 0, net_req_tx4_2['Net req'])
#net_req_tx4.drop(labels=['comming', 'OHTX4'],axis=1,inplace= True)
#net_req_tx4_2.drop(labels=['OHTX4'],axis=1,inplace= True)
#net_req_tx4=net_req_tx4.drop_duplicates() 
net_req_tx4_2.head()

,#*Zebra_x,*Supplier ID,*Zebra PN,*Zebra Site_x,*Transaction Type,Quantity_x,*Date_x,month,SO_x,FCST or SO,...,Fcst_before_july,*Date_y,*Zebra Site_y,OH_beforeuly,SO_y,IRS,Demand_y,Quantity_y,OHTX4,Net req
0,Zebra,12272,CBA-UF6-C12ZAR,TX4,GROSSFRCTWK,3,2022-06-13,beforejuly,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0
1,Zebra,12272,CBA-UF6-C12ZAR,TX4,GROSSFRCTWK,3,2022-06-05,beforejuly,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0
2,Zebra,12272,CBL-TC7X-USB1-01,TX4,GROSSFRCTWK,3,2022-06-13,beforejuly,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0
3,Zebra,12272,SAC-ET5X-4PPK1-01,TX4,GROSSFRCTWK,3,2022-06-05,beforejuly,0.0,3.0,...,0.0,2022-06-05,TX2,9.0,0.0,0.0,0.0,0.0,9.0,0.0
4,Zebra,12272,CBA-UF6-C12ZAR,TX4,GROSSFRCTWK,3,2022-06-06,beforejuly,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0


In [155]:
forecast_afterjuly=net_req_tx4_2.drop(labels=['#*Zebra_x','#*Zebra_x','*Zebra Site_x','*Zebra Site_y',
                                                    '*Transaction Type','month'],axis=1)

#order by date, #order rows
forecast_afterjuly.sort_values(by=['*Supplier ID','*Zebra PN','*Date_x'], ascending=True, inplace=True)
forecast_afterjuly['Cummulative_fcst'] = forecast_afterjuly.groupby(['*Zebra PN','*Supplier ID'])['FCST or SO'].cumsum(axis = 0)
forecast_afterjuly

,*Supplier ID,*Zebra PN,Quantity_x,*Date_x,SO_x,FCST or SO,Plant Safety Stock,Demand_x,#*Zebra_y,Fcst_before_july,*Date_y,OH_beforeuly,SO_y,IRS,Demand_y,Quantity_y,OHTX4,Net req,Cummulative_fcst
7,12272,CBA-U21-S07ZBR,3,2022-06-05,0.0,3.0,0.0,3.0,Zebra,0.0,2022-06-05,44.0,0.0,0.0,0.0,0.0,44.0,0.0,3.0
14,12272,CBA-U43-S07ZAR,3,2022-06-05,0.0,3.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0,3.0
13,12272,CBA-U43-S07ZAR,3,2022-06-06,0.0,3.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0,6.0
10,12272,CBA-U43-S07ZAR,3,2022-06-13,0.0,3.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0,9.0
5,12272,CBA-U43-S07ZAR,3,2022-06-24,0.0,3.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,38496,SG-ET4X-10EXOSKL1-01,3,2022-06-24,0.0,3.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0,12.0
198,38496,SG-ET4X-8EXOSKL1-01,3,2022-06-05,0.0,3.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0,3.0
202,38496,SG-ET4X-8EXOSKL1-01,3,2022-06-06,0.0,3.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0,6.0
199,38496,SG-ET4X-8EXOSKL1-01,3,2022-06-13,0.0,3.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0,9.0


In [157]:
#min between cummulative-oh and quantity
forecast_afterjuly['cumm-oh']=forecast_afterjuly['Cummulative_fcst']-forecast_afterjuly['OHTX4']
forecast_afterjuly['min']=forecast_afterjuly[['cumm-oh','FCST or SO']].min(axis=1)    

#when we actually need forecast(oh+wip) not enough to cover forecast see if net req has covered so far in date forecast
#if its ccovered then net req is 0 in that date if not send fcst need for that date
#if oh+wip has covered fcst then 0 for all dates
forecast_afterjuly['Net_req1']=np.where(forecast_afterjuly['Net req']>0,np.where(forecast_afterjuly['Cummulative_fcst']<forecast_afterjuly['OHTX4'],
                                                                               0,forecast_afterjuly['min']),0)            

In [158]:
forecast_afterjuly.head()

,*Supplier ID,*Zebra PN,Quantity_x,*Date_x,SO_x,FCST or SO,Plant Safety Stock,Demand_x,#*Zebra_y,Fcst_before_july,...,SO_y,IRS,Demand_y,Quantity_y,OHTX4,Net req,Cummulative_fcst,cumm-oh,min,Net_req1
7,12272,CBA-U21-S07ZBR,3,2022-06-05,0.0,3.0,0.0,3.0,Zebra,0.0,...,0.0,0.0,0.0,0.0,44.0,0.0,3.0,-41.0,-41.0,0.0
14,12272,CBA-U43-S07ZAR,3,2022-06-05,0.0,3.0,0.0,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,3.0,3.0,3.0,3.0,3.0
13,12272,CBA-U43-S07ZAR,3,2022-06-06,0.0,3.0,0.0,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,3.0,6.0,6.0,3.0,3.0
10,12272,CBA-U43-S07ZAR,3,2022-06-13,0.0,3.0,0.0,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,3.0,9.0,9.0,3.0,3.0
5,12272,CBA-U43-S07ZAR,3,2022-06-24,0.0,3.0,0.0,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,3.0,12.0,12.0,3.0,3.0


# RUN IF RM WIP AVailable juts consider RMA, WIP for TX4 even beforejuly

In [ ]:
#sum just the measures to extract from forecast,besides newOH uploaded in TX4S #edit
rest_to_forecast= zcmuploads[(zcmuploads['*Transaction Type']=='INTERNALREQDLY') | (zcmuploads['*Transaction Type'] == 'RMADLY')]
rest_to_fcst_piv=rest_to_forecast.pivot_table(index=('*Zebra PN','*Supplier ID'), columns='month', aggfunc='sum').fillna(0)
print(len(rest_to_fcst_piv))
#zcmuploadspiv['*Transaction Type'] ='ONHANDWK'
rest_to_fcst_piv.head()

#add qty to substract(sum RMA, Intransit)
net_req_tx4= pd.merge(forecast_tx4_piv, rest_to_fcst_piv, on =['*Zebra PN','*Supplier ID'],how='left')
#net_req_tx4.drop(labels=[('Quantity', 'beforejuly')],axis=1,inplace= True)
net_req_tx4.rename(columns={('Quantity', 'afterjuly'):'comming'}, inplace= True)

net_req_tx4

In [165]:
strings = time.strftime("%m.%d.%Y")
stringsnr= 'Net_Requirements ' + strings 
b=("%s.xlsx" % stringsnr)

with pd.ExcelWriter(b) as writer:  
       net_req_tx4_1.to_excel(writer, index= False)

# OLD

In [ ]:
#Net req to upload
#what NET REQ we will upload? those PN in after july, before july wias already considered 
net_req_tx4_0= zcmuploads[(zcmuploads['month']=='afterjuly')]
#just one part umber per month 
net_req_tx4_0.drop(labels=['*Date','*Transaction Type','month','Quantity'],axis=1, inplace = True)
net_req_tx4_0 = net_req_tx4_0.drop_duplicates()
net_req_tx4_0
#zcm_onhand_dly13.drop(labels=['*Date','month'],axis=1,inplace= True)

#add forecast per row
net_req_tx4= pd.merge(net_req_tx4_0, forecast_afterjuly, on =['*Zebra PN','*Supplier ID'],how='left')
net_req_tx4.rename(columns={'Quantity': 'Forecast'},inplace=True)
net_req_tx4

#add qty to substract(sum RMA, Intransit)
net_req_tx4= pd.merge(net_req_tx4, rest_to_fcst_piv, on =['*Zebra PN','*Supplier ID'],how='left')
net_req_tx4.drop(labels=[('Quantity', 'beforejuly')],axis=1,inplace= True)
net_req_tx4.rename(columns={('Quantity', 'afterjuly'):'comming'}, inplace= True)

net_req_tx4

In [253]:
#add OHTX4 afterjuly
oh_tx4 = zcm_net_req1
oh_tx4.drop(labels=['*Date','#Customer_Name','*Zebra Site','*Transaction Type'],axis=1, inplace= True)

net_req_tx4= pd.merge(net_req_tx4,oh_tx4, on =['*Zebra PN','*Supplier ID'],how='left')
#net_req_tx4['comming'].fillna(0, inplace=True)
#net_req_tx4['oh+wip+all'] = net_req_tx4['comming']+net_req_tx4['OHTX4']
net_req_tx4['oh+wip+all'] = net_req_tx4['OHTX4']
net_req_tx4['Net req'] = net_req_tx4['Forecast'] - net_req_tx4['oh+wip+all']
net_req_tx4

net_req_tx4['Net req'] = np.where(net_req_tx4['Net req'] < 0, 0, net_req_tx4['Net req'])
#net_req_tx4.drop(labels=['comming', 'OHTX4'],axis=1,inplace= True)
net_req_tx4.drop(labels=['OHTX4'],axis=1,inplace= True)
net_req_tx4=net_req_tx4.drop_duplicates() 
net_req_tx4
#-------------
net_req_tx4_date=zcm_onhand_dly[zcm_onhand_dly['month']=='afterjuly']
net_req_tx4_date['Cummulative_fcst'] = net_req_tx4_date.groupby(['*Zebra PN','*Supplier ID'])['Quantity'].cumsum(axis = 0)

net_req_tx41=net_req_tx4.drop(labels = ['#Customer_Name','#Customer_Name','*Zebra Site','Supplier PN'], axis=1)
net_req_tx4_date

net_req_tx4_date1 = pd.merge(net_req_tx4_date,net_req_tx41, on =['*Zebra PN','*Supplier ID'],how='left')
net_req_tx4_date1.tail()

net_req_tx4_date1['Net_req1']=np.where(net_req_tx4_date1['Net req']>0,np.where(net_req_tx4_date1['Cummulative_fcst']<net_req_tx4_date1['Net req'],
                                                                               0,net_req_tx4_date1['Quantity']),0)
net_req_tx4_date1.drop(labels=['month','Forecast','oh+wip+all','Net req'],axis=1,inplace = True)
net_req_tx4_date1.tail()

,#Customer_Name,*Supplier ID,*Zebra PN,*Zebra Site,Quantity,*Date,*Transaction Type,month,Cummulative_fcst
6269,Zebra,25738,P1070813-001,TX4,6.0,2022-08-29,GROSSFRCTWK,afterjuly,6.0
6270,Zebra,25738,P1070813-001,TX4,5.0,2022-09-05,GROSSFRCTWK,afterjuly,11.0
6271,Zebra,25738,P1070813-001,TX4,5.0,2022-09-12,GROSSFRCTWK,afterjuly,16.0
6272,Zebra,25738,P1070813-001,TX4,5.0,2022-09-19,GROSSFRCTWK,afterjuly,21.0
6273,Zebra,25738,P1070813-001,TX4,5.0,2022-09-26,GROSSFRCTWK,afterjuly,26.0
...,...,...,...,...,...,...,...,...,...
211205,Zebra,24365,MPACT-SB1100-01-WR,TX2,25410.0,2022-08-15,GROSSFRCTWK,afterjuly,289763.0
211206,Zebra,24365,MPACT-SB1100-01-WR,TX2,2950.0,2022-09-12,GROSSFRCTWK,afterjuly,292713.0
211207,Zebra,24365,MPACT-SB1100-01-WR,TX2,582.0,2022-09-26,GROSSFRCTWK,afterjuly,293295.0
211208,Zebra,24365,MPACT-SB1100-01-WR,TX2,1761.0,2023-07-03,GROSSFRCTWK,afterjuly,295056.0


# OLD

In [132]:
#TX4 OH will be available from 8 jul, too add to oh (tx2 after july)